In [16]:
%matplotlib widget
import matplotlib.pyplot as plt

In [17]:
from gen_data import get_data
d = get_data(25)
m = d[5]

In [18]:
fig, ax = plt.subplots()
control = float(m.control)
t = m.time
z = m.values
(ln,) = ax.plot(t, z, label=f"C: {control:.1f}")
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [21]:
def plot_one(ax, m):
    # pull what we want out of the xarray
    control = float(m.control)
    t = m.time
    z = m.values

    (ln,) = ax.plot(t, z, label=f"C: {control:.1f}")

    return {"raw": ln}


fig, ax = plt.subplots()
plot_one(ax, d[0])
plot_one(ax, d[5])
plot_one(ax, d[-1])
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [22]:
def plot_one(ax, m, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.time
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")

    return {"raw": ln}


fig, ax = plt.subplots()
plot_one(ax, d[0])
plot_one(ax, d[10], offset=4)
plot_one(ax, d[20], offset=8)
ax.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [23]:
from gen_data import fit

In [28]:
fit_vals = fit(d[6])
print(repr(fit_vals))
print(fit_vals)
print(fit_vals._repr_latex_())
fit_vals

Params(A=2.0014400039521316, zeta=0.02658800859092437, omega=3.563013819697966, phi=1.5653506273195317)
(A=2.00, ζ=0.03, ω0=3.56, φ=1.57)
$(A=2.00, \zeta=0.03, \omega_0=3.56,\varphi=1.57)$


Params(A=2.0014400039521316, zeta=0.02658800859092437, omega=3.563013819697966, phi=1.5653506273195317)

In [29]:
fig, ax = plt.subplots()
ax.plot(m.time, fit_vals.sample(m.time), label=fit_vals._repr_latex_(), color='k')
plt.gca().set_title(
     r"$A e^{-\zeta\omega_0t} \sin\left(\sqrt{1 - \zeta^2}\omega_0t + \varphi\right)$",
     usetex=True,
)

plt.legend()
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'displacement (mm)')

In [33]:
def plot_one(ax, m, popt, offset=0):
    # pull what we want out of the xarray
    control = float(m.coords["control"])
    t = m.coords["time"]
    z = m.values

    (ln,) = ax.plot(t, z + offset, label=f"C: {control:.1f}")
    (fit,) = ax.plot(t, popt.sample(t) + offset, color="k")
    ann = ax.annotate(
        f"$\\zeta={popt.zeta:.2g}$, $\\omega_0={popt.omega:.2f}$",
        # units are (axes-fraction, data)
        xy=(0.95, offset + 0.5),
        xycoords=ax.get_yaxis_transform(),
        # set the text alignment
        ha="right",
        va="bottom",
    )
    return {"raw": ln, "fit": fit, "annotation": ann}


In [36]:
fig, ax = plt.subplots()
plot_one(ax, d[0], fit(d[0]))
plot_one(ax, d[10], fit(d[10]), offset=4)
plot_one(ax, d[20], fit(d[20]), offset=8)
plot_one(ax, d[24], fit(d[24]), offset=12)
ax.legend(ncol=3, loc="upper center")
ax.set_xlabel("time (ms)")
ax.set_ylabel("displacement (mm)")
plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
def plot_several(ax, d, fits):
    out = []

    for j, (m, popt) in enumerate(zip(d, fits)):
        arts = plot_one(ax, m, popt, offset=3.75 * j)
        out.append(arts)

    ax.set_xlabel("time (ms)")
    ax.set_ylabel("displacement (mm)")

    return out


In [38]:
fig, ax = plt.subplots()
indx = [0, 5, 15, 24]
plot_several(ax, d[indx], [fit(d[i]) for i in indx])
ax.legend(ncol=3, loc="upper center")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …